# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from apikey import api_key 
g_key=api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#importing csv file from weather

In [3]:
weather="weather_lat_data.csv"
weather_df=pd.read_csv(weather)
weather_df

,Unnamed: 0,Latitude,Longitude,Temp (F),Humidity (%),Windspeed (mph),Cloudiness (%),City
0,0,65.6145,-37.6368,24.80,42,24.16,90,Tasiilaq
1,1,-0.8833,131.2500,83.35,65,6.53,62,Sorong
2,2,-17.8333,-149.2667,84.09,74,5.75,40,Teahupoo
3,3,45.1667,10.7833,37.38,80,3.44,0,Province of Mantua
4,4,71.6872,128.8694,-28.12,85,3.91,100,Tiksi
...,...,...,...,...,...,...,...,...
550,550,-32.2500,148.6167,82.40,26,6.91,40,Dubbo
551,551,53.9667,121.9667,1.22,85,3.02,100,Yerofey Pavlovich
552,552,-14.3047,-43.7650,68.94,81,4.97,99,Carinhanha
553,553,45.0134,38.3379,36.01,81,13.22,92,Mingrelskaya


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#From the previous dataset, extracting coordinates and humidity


In [5]:
gmaps.configure(g_key)

locations = weather_df[["Latitude","Longitude"]].astype(float)

rel_hum = weather_df["Humidity (%)"].astype(float)
max_hum=weather_df["Humidity (%)"].max().astype(float)

In [6]:
#creating heat map with the entire dataset with weight on humidity and all locations
fig = gmaps.figure(center=(0,12),zoom_level=2)

hum_layer=gmaps.heatmap_layer(locations, weights=rel_hum,
                              dissipating=False, max_intensity=max_hum,
                              point_radius = 2)

fig.add_layer(hum_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:

nice_w_df = weather_df.rename(columns={"Temp (F)":"Temp","Humidity (%)":"Humidity","Windspeed (mph)":"ws","Cloudiness (%)":"Cloudiness"})

nice_w_df["lat,lon"] = nice_w_df["Latitude"].map(str)+(',') + nice_w_df["Longitude"].map(str)

#Filtering by weather conditions: 72<T=<80, wind speed below 6 mph, up to 10% of cloudiness and humidity below 80%

nice_w_df=nice_w_df[(nice_w_df.Temp>72)&(nice_w_df.Temp<=85)&(nice_w_df.ws<6)&(nice_w_df.Cloudiness<10)&(nice_w_df.Humidity<80)]
nice_w_df


,Unnamed: 0,Latitude,Longitude,Temp,Humidity,ws,Cloudiness,City,"lat,lon"
60,60,-21.3585,55.7679,80.33,51,5.75,0,Saint-Philippe,"-21.3585,55.7679"
87,87,20.8624,92.3058,82.18,52,5.91,4,Teknāf,"20.8624,92.3058"
157,157,-14.9003,50.2788,80.19,77,3.83,9,Antalaha,"-14.9003,50.2788"
166,166,31.0775,75.3372,82.00,40,0.51,0,Shāhkot,"31.0775,75.3372"
278,278,21.8500,84.0333,82.40,51,4.61,0,Jharsuguda,"21.85,84.0333"
349,349,26.6921,67.6133,83.80,13,1.41,0,Johi,"26.6921,67.6133"
352,352,19.3003,97.9685,77.00,50,4.27,0,Mae Hong Son,"19.3003,97.9685"
371,371,19.4833,75.3833,80.60,28,4.61,0,Paithan,"19.4833,75.3833"
414,414,25.6667,85.8333,81.12,16,3.27,0,Dalsingh Sarai,"25.6667,85.8333"
493,493,26.4500,74.6333,81.99,8,4.68,0,Ajmer,"26.45,74.6333"


In [8]:
#creating a new heat map with the filtered locations of perfect weather conditions

nice_w_locations = nice_w_df[["Latitude","Longitude"]].astype(float)
rel_hum = nice_w_df["Temp"].astype(float)

fig2 = gmaps.figure(center=(0,12),zoom_level=2)

hum_layer2=gmaps.heatmap_layer(nice_w_locations, weights=rel_hum,
                              dissipating=False, max_intensity=80,
                              point_radius = 2)

fig2.add_layer(hum_layer2)

fig2

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#creating empty lists for further use

In [10]:
hotel_name=[]
hotel_loc=[]
hotel_loc1=[]
hotel_country=[]
hotel_address=[]

In [11]:
# Finding hotels in a radius of 5 Km using filtered coordinates for  perfect weather
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_type = "hotel"
target_radius = 5000

nice_loc = nice_w_df[['lat,lon']] #extracting the Latitudes and Longitudes as strings to be used in the location parameter
nice_loc=nice_loc.values.tolist()



In [12]:
#obtaining responses of hotel names and hotel coordinates
i=0
j=[]
for coord in nice_loc:
    params = {"location": coord,
             "keyword": target_search,
             "radius": target_radius,
             "type": target_type,
             "key": g_key}
    
   
    response = requests.get(base_url, params=params)
    response_js=response.json()
    
    try:
        hotel_name.append(response_js['results'][0]['name'])
        hotel_loc.append(response_js['results'][0]['geometry']['location'])
        hotel_loc1.append(response_js['results'][0]['geometry']['location'])  
       
        
    except IndexError: #not all the coordinates can find a hotel in 5km radius
        i=i+1
   
print(f'There are not hotels in {i} cities, you might want to expand your search radius')

There are not hotels in 2 cities, you might want to expand your search radius


In [13]:
hotel_name=pd.DataFrame(hotel_name) #data frame of found hotels. Note that the # of rows is less than the locations. This is because cities are droppout
hotel_name

,0
0,Les Embruns Du Baril
1,Central Resort Ltd.
2,Belle rose beach Hotel
3,Hotel Orchard
4,Imperial Mae Hong Son Hotel
5,OYO 61310 Manas Resort
6,ARC Hotel
7,Hotel Atlantica ajmer
8,LODGE PALMAE
9,Raja Hotel


In [14]:
hotel_loc=pd.DataFrame(hotel_loc) #dataframe of coordinates for all the hotels
hotel_loc

,lat,lng
0,-21.369201,55.727949
1,20.848162,92.273031
2,-14.914644,50.292982
3,21.861538,83.999194
4,19.279987,97.958842
5,19.479049,75.378897
6,25.665324,85.839670
7,26.467590,74.606302
8,-21.296958,55.499267
9,25.826867,69.375562


In [15]:
hotels_info_df= pd.concat([hotel_name,hotel_loc], axis=1) #creating a df with hotel names and coordinates
hotels_info_df=hotels_info_df.rename(columns={0:'name'})
hotels_info_df

,name,lat,lng
0,Les Embruns Du Baril,-21.369201,55.727949
1,Central Resort Ltd.,20.848162,92.273031
2,Belle rose beach Hotel,-14.914644,50.292982
3,Hotel Orchard,21.861538,83.999194
4,Imperial Mae Hong Son Hotel,19.279987,97.958842
5,OYO 61310 Manas Resort,19.479049,75.378897
6,ARC Hotel,25.665324,85.839670
7,Hotel Atlantica ajmer,26.467590,74.606302
8,LODGE PALMAE,-21.296958,55.499267
9,Raja Hotel,25.826867,69.375562


In [16]:
markers = gmaps.marker_layer(hotel_loc) #adding markers to the perfect weather map
fig2.add_layer(markers)
fig2

Figure(layout=FigureLayout(height='420px'))

In [17]:
hotel_loc1 #auxiliar df for hotel coordinates
hotel_loc1=pd.DataFrame(hotel_loc1)
hotel_loc1

,lat,lng
0,-21.369201,55.727949
1,20.848162,92.273031
2,-14.914644,50.292982
3,21.861538,83.999194
4,19.279987,97.958842
5,19.479049,75.378897
6,25.665324,85.839670
7,26.467590,74.606302
8,-21.296958,55.499267
9,25.826867,69.375562


In [18]:
#concatenating lat and lon to build urls
hotel_coord_df = hotel_loc1
hotel_coord_df['lat,lon'] = hotel_coord_df["lat"].map(str)+(',') + hotel_coord_df["lng"].map(str)
hotel_coord=hotel_coord_df['lat,lon'].values.tolist()

In [19]:
#building urls, obtaining responses of countries, using a loop through the hotels coordinates


for coord in hotel_coord:
    
    query_url_country=f'https://maps.googleapis.com/maps/api/geocode/json?latlng={coord}&location_type=APPROXIMATE&result_type=country&key={g_key}'
    response_country = requests.get(query_url_country)
    response_country_js=response_country.json()
    hotel_country.append(response_country_js['results'][0]['address_components'][0]['long_name'])


hotel_country
hotel_country_df=pd.DataFrame(hotel_country)


In [20]:
hotels_info_df["country"]= hotel_country_df #appending the countries to the hotels info df
hotels_info_df

,name,lat,lng,country
0,Les Embruns Du Baril,-21.369201,55.727949,Réunion
1,Central Resort Ltd.,20.848162,92.273031,Bangladesh
2,Belle rose beach Hotel,-14.914644,50.292982,Madagascar
3,Hotel Orchard,21.861538,83.999194,India
4,Imperial Mae Hong Son Hotel,19.279987,97.958842,Thailand
5,OYO 61310 Manas Resort,19.479049,75.378897,India
6,ARC Hotel,25.665324,85.839670,India
7,Hotel Atlantica ajmer,26.467590,74.606302,India
8,LODGE PALMAE,-21.296958,55.499267,Réunion
9,Raja Hotel,25.826867,69.375562,Pakistan


In [21]:
# trying to pull the address...the dictionaries info varies randomly. Assigned an arbitrary key to the city and called it address.
for coord in hotel_coord:
    
    query_url_address=f'https://maps.googleapis.com/maps/api/geocode/json?latlng={coord}&key={g_key}'
    response_address = requests.get(query_url_address)
    response_address_js=response_address.json()
    hotel_address.append(response_address_js['results'][0]['address_components'][0]['long_name']) 


In [22]:
hotel_address_df=pd.DataFrame(hotel_address)
hotel_address_df=hotel_address_df.rename(columns={0:"address"})
hotel_address_df

,address
0,54
1,Z1098
2,Double Voie Belle Rose
3,Sambalpur - Rourkela Highway
4,108
5,Main Road
6,Gudari Road
7,950
8,48
9,Sanghar Road


In [23]:
hotels_info_df["address"]= hotel_address_df #expanded hotels info df with "address"
hotels_info_df

,name,lat,lng,country,address
0,Les Embruns Du Baril,-21.369201,55.727949,Réunion,54
1,Central Resort Ltd.,20.848162,92.273031,Bangladesh,Z1098
2,Belle rose beach Hotel,-14.914644,50.292982,Madagascar,Double Voie Belle Rose
3,Hotel Orchard,21.861538,83.999194,India,Sambalpur - Rourkela Highway
4,Imperial Mae Hong Son Hotel,19.279987,97.958842,Thailand,108
5,OYO 61310 Manas Resort,19.479049,75.378897,India,Main Road
6,ARC Hotel,25.665324,85.839670,India,Gudari Road
7,Hotel Atlantica ajmer,26.467590,74.606302,India,950
8,LODGE PALMAE,-21.296958,55.499267,Réunion,48
9,Raja Hotel,25.826867,69.375562,Pakistan,Sanghar Road


In [24]:

#adding info boxes with hotels information to the perfect weather map
#################
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{address}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
########################

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_info_df.iterrows()]

markers = gmaps.marker_layer(hotel_loc,info_box_content=hotel_info)
fig2.add_layer(markers)


In [25]:
fig2

Figure(layout=FigureLayout(height='420px'))